# JSON Files

After CSV, the [JSON](https://en.wikipedia.org/wiki/JSON) file type is one of the most popular to encode data. JSON files look and mostly act like python dictionaries, with a few quirks. Here is an example of a valid JSON file:


```
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 27,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    }
  ],
  "children": [],
  "spouse": null
}
```

Which we could read into python with the `json` core library:

In [1]:
import json

d = json.loads("""
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 27,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    }
  ],
  "children": [],
  "spouse": null
}
""")

d

{'firstName': 'John',
 'lastName': 'Smith',
 'isAlive': True,
 'age': 27,
 'address': {'streetAddress': '21 2nd Street',
  'city': 'New York',
  'state': 'NY',
  'postalCode': '10021-3100'},
 'phoneNumbers': [{'type': 'home', 'number': '212 555-1234'},
  {'type': 'office', 'number': '646 555-4567'}],
 'children': [],
 'spouse': None}

In [2]:
d.keys()

dict_keys(['firstName', 'lastName', 'isAlive', 'age', 'address', 'phoneNumbers', 'children', 'spouse'])

In [3]:
d['phoneNumbers']

[{'type': 'home', 'number': '212 555-1234'},
 {'type': 'office', 'number': '646 555-4567'}]

In [4]:
with open('data_file.json','w') as write_file:
    json.dump(data, write_file)

NameError: name 'data' is not defined

JSON supports key-value types, arrays, numbers, booleans, strings and some other features. It has quirks (read the specification!) like:

- Strings have to be denoted with `"`, not `'`

- You can't have commas on the last element in a container like in python

### Recipe Database

Vectorized string operations become most useful in the process of cleaning up messy, real-world data.

Here I'll walk through an example of that, using an open recipe database compiled from various sources on the Web.
Our goal will be to parse the recipe data into ingredient lists, so we can quickly find a recipe based on some ingredients we have on hand.

This database is about 30 MB compress in the `gzip` format, and 140MB uncompressed.

The database is in JSON format, so we will try `pd.read_json` to read it after decompressing it:

In [2]:
import gzip
import numpy as np
import pandas as pd

with gzip.open("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/recipe.json.gz",'r') as f:
    try:
        recipes = pd.read_json(f)
    except ValueError as e:
        print("ValueError:", e)

ValueError: Trailing data


Oops! We get a ``ValueError`` mentioning that there is "trailing data."
Searching for the text of this error on the Internet, it seems that it's due to using a file in which *each line* is itself a valid JSON, but the full file is not.
Let's check if this interpretation is true:

In [5]:
with gzip.open("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/recipe.json.gz",'r') as f:
    line = f.readline()
pd.read_json(line).shape

TypeError: Expected file path name or file-like object, got <class 'bytes'> type

Yes, apparently each line is a valid JSON, so we'll need to string them together.
One way we can do this is to actually construct a string representation containing all these JSON entries, and then load the whole thing with ``pd.read_json``:

In [6]:
# read the entire file into a Python array
with gzip.open("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/recipe.json.gz", 'r') as f:
    # Extract each line
    data = (line.strip().decode() for line in f)
    # Reformat so each line is the element of a list
    data_json = f"[{','.join(data)}]"
# read the result as a JSON
recipes = pd.read_json(data_json)

In [7]:
recipes.shape

(173278, 17)

We see there are nearly 200,000 recipes, and 17 columns.
Let's take a look at one row to see what we have:

In [11]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

There is a lot of information there, but much of it is in a very messy form, as is typical of data scraped from the Web.
In particular, the ingredient list is in string format; we're going to have to carefully extract the information we're interested in.
Let's start by taking a closer look at the ingredients:

In [12]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

The ingredient lists average 250 characters long, with a minimum of 0 and a maximum of nearly 10,000 characters!

Just out of curiousity, let's see which recipe has the longest ingredient list:

In [13]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

# JSON fields

Sometimes, values in columns are encoded as JSON fields.

One example is in the [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) from IMDB. For convenience, I've rehosted the data in a Google Drive which we can read with the following methods:

In [4]:
import pandas as pd
import json
movies_url = {
"movies_metadata": "1RLvh6rhzYiDDjPaudDgyS9LmqjbKH-wh",
"keywords": "1YLOIxb-EPC_7QpkmRqkq9E6j7iqmoEh3",
"ratings": "1_5HNurSOMnU0JIcXBJ5mv1NaXCx9oCVG",
"credits": "1bX9othXfLu5NZbVZtIPGV5Hbn8b5URPf",
"ratings_small": "1fCWT69efrj4Oxdm8ZNoTeSahCOy6_u6w",
"links_small": "1fh6pS7XuNgnZk2J3EmYk_9jO_Au_6C15",
"links": "1hWUSMo_GwkfmhehKqs8Rs6mWIauklkbP",
}

def read_gdrive(url):
    """
    Reads file from Google Drive sharing link
    """
    path = 'https://drive.google.com/uc?export=download&id='+url
    return pd.read_csv(path)

df = read_gdrive(movies_url["movies_metadata"])
df

C:\Users\Simona\AppData\Local\Temp\ipykernel_13680\1319032021.py:18: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


Here, a few columns, like `genres` and `belongs_to_collection` are JSON types. Let's look at one:

In [2]:
df.genres.iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

# Using df.apply on a columns

Seems to be a list of dicts. Let's say we want a list of genre names.

We could try to parse it with `.str.method()` chains of calls, but we can also use `df.apply` to apply a method to each row.

Let's try to load the strings on each row as a list object through `json.loads`:

In [5]:
df.genres.apply(json.loads)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

So we have a decode error because the characters are `'` instead of `"`. Let's fix it and apply:

In [6]:
(df.genres
   .str.replace("'", '"')
   .apply(json.loads)
)[1]

[{'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 10751, 'name': 'Family'}]

We can use `.apply` to parse each dict now:

In [ ]:
(df.genres
   .str.replace("'", '"')
   .apply(json.loads)
   .apply(lambda row : [d['name'] for d in row])
)

And we have our result!

Note that we could have put all the logic in a single `.apply` but this is often slower (more python code is executed):

In [ ]:
def unpack_row(r):
    r = r.replace("'", '"')
    r = json.loads(r) # cast string to list
    return [d['name'] for d in r]

df.genres.apply(unpack_row)

Which you use is a matter of personal choice and performance requirements of your data pipeline.

We could also parse all genres into a one-hot encoded matrix with a few more steps:

In [ ]:
(df.genres
   # Could be optimized
   .str.replace("'", '"')
   .apply(json.loads)
   .apply(lambda row : [d['name'] for d in row])
   .astype(str)
   # Could be done in a single regex
   .str.replace("[", "")
   .str.replace("]", "")
   .str.get_dummies(',')
)

This isn't particularly efficient or elegant, but it works!

# The magic of `eval`

Another way to load these fields is to use python's `eval` functionality, which loads a string as code:

In [ ]:
eval("print('hi')")
eval("['a', 'v', 'c']")

This is less stringent than loadins a string from JSON for malformed data:

In [ ]:
(df.genres
   # Read as python code into list of dicts
   .apply(eval)
   .apply(lambda row : [d['name'] for d in row])
   .astype(str)
   .str.replace("[", "")
   .str.replace("]", "")
   .str.get_dummies(',')
)